##  How to use in Fabric? 

This notebook reads the raw data from Bronze Lakehouse and writes transformed data to Silver Lakehouse.

1. Create your Bronze and Silver Lakehouses in your fabric environment. 
2. Create the mds-mapping-fabric.ipynb notebook in your fabric environment.
3. Upload the raw data, the conceptual_mapping.xlsx, Appendix/Entities into Bronze Lakehouse. Folder structire should look like this:
    - BronzeLakehouse
        * Files
            - raw
                * Bakery CSVs
            - config
                * conceptual_mapping.xlsx
            - Appendix
                * Entities/
4. Add Bronze and Silver Lakehouses to your notebook.
5. Add create_config.py and transform.py as Built-in resources.
7. Change *silver_base_path* to your absolute path of Silver Lakehouse.
8. Change *bronze_base_path* to your absolute path of Bronze Lakehouse.
9. Run the notebook.

## Imports

In [ ]:
import pandas as pd
import os
from notebookutils import mssparkutils
import builtin.transform as transform
import builtin.create_config as create_config

## Input and Output Directories

*EDIT THE SILVER AND BRONZE BASE PATHS*

In [ ]:
bronze_mount_name = "/Bronze"
silver_mount_name = "/Silver"
bronze_base_path = "<ABSOLUTE PATH OF BRONZE LAKEHOUSE>"
silver_base_path = "<ABSOLUTE PATH OF SILVER LAKEHOUSE>"
mssparkutils.fs.mount(bronze_base_path, bronze_mount_name)
mssparkutils.fs.mount(silver_base_path, silver_mount_name)
mount_points = mssparkutils.fs.mounts()

bronze_path = next((mp["localPath"] for mp in mount_points if mp["mountPoint"] == bronze_mount_name), None)
silver_path = next((mp["localPath"] for mp in mount_points if mp["mountPoint"] == silver_mount_name), None)

# INPUT DATA FOLDER
INPUT_DATA_FOLDER = f"{bronze_path}/Files/raw" 

# DTDL FOLDER
ENTITY_PATH = f"{bronze_path}/Files/Appendix/Entities/"

# CONFIG LOCATION
CONFIG_FOLDER = f"{bronze_path}/Files/config"

# OUTPUT DATA FOLDER
OUTPUT_DATA_FOLDER = f"{silver_path}/Files"

In [ ]:
EXCEL_MAPPING_FILE = f"{CONFIG_FOLDER}/conceptual_mapping.xlsx"
OUTPUT_MAPPING_FOLDER = f"{OUTPUT_DATA_FOLDER}/mapping"
CONFIG_FILE = f"{CONFIG_FOLDER}/config.json"


# CREATE THE OUTPUT FOLDER IF NOT EXISTING
if not os.path.exists(OUTPUT_MAPPING_FOLDER):
    os.makedirs(OUTPUT_MAPPING_FOLDER)
if not os.path.exists(CONFIG_FOLDER):
    os.makedirs(CONFIG_FOLDER)

## Creating Configuration File

In [ ]:
config_dictionary = create_config.create_entity_mappings_config(EXCEL_MAPPING_FILE)
config_dictionary = create_config.create_relationship_mappings_config(EXCEL_MAPPING_FILE, config_dictionary)
create_config.write_config_json(config_dictionary, CONFIG_FILE)

### Extracting Variables from Config

In [ ]:
df = pd.read_json(CONFIG_FILE, typ="series")
ENTITY_LIST = df['entity_list']
RELATIONSHIP_LIST = df['relationship_list']

## Transform the raw data

In [ ]:
transform.cache_entity_metadata(ENTITY_LIST, ENTITY_PATH)
transform.write_entity_data(INPUT_DATA_FOLDER, OUTPUT_DATA_FOLDER, OUTPUT_MAPPING_FOLDER, ENTITY_LIST, RELATIONSHIP_LIST)